# 计算区域灌溉能力


In [1]:
import os
import arcpy
from arcpy.sa import *

In [ ]:
# 设置环境参数
arcpy.env.workspace = "C:/path/to/your/workspace"
arcpy.env.overwriteOutput = True

# 检查Spatial Analyst扩展模块
arcpy.CheckOutExtension("Spatial")

In [ ]:
# 导入DEM、土壤类型、河流和沟渠数据、降雨量数据、蒸散量数据
dem = "DEM.tif"
soil_type = "soil_type.tif"
rivers_shp = "rivers.shp"
canals_shp = "canals.shp"
precipitation = "precipitation.tif"
evapotranspiration = "evapotranspiration.tif"


In [ ]:
# 填平DEM数据中的凹陷
filled_dem = "filled_dem.tif"
arcpy.gp.Fill_sa(dem, filled_dem)

# 计算坡度
slope = "slope.tif"
arcpy.gp.Slope_sa(filled_dem, slope, "DEGREE")

# 计算流向
flow_direction = "flow_direction.tif"
arcpy.gp.FlowDirection_sa(filled_dem, flow_direction)

# 计算流积量
flow_accumulation = "flow_accumulation.tif"
arcpy.gp.FlowAccumulation_sa(flow_direction, flow_accumulation)


In [ ]:
# 合并河流和沟渠图层
combined_features = "combined_features.shp"
arcpy.Merge_management([rivers_shp, canals_shp], combined_features)

# 计算每个栅格单元到最近河流和沟渠的距离
distance_to_water_source = "distance_to_water_source.tif"
arcpy.gp.EucDistance_sa(combined_features, distance_to_water_source)

# 计算每个栅格单元的高程差
elevation_difference = "elevation_difference.tif"
arcpy.gp.EucAllocation_sa(dem, elevation_difference, "", combined_features, "VALUE", "", "NO_NEAREST")

# 计算距离和高程差的加权组合，以反映实际可用性
weighted_distance = "weighted_distance.tif"
expression = "Raster('{}') + 0.5 * Raster('{}')".format(distance_to_water_source, elevation_difference)
arcpy.gp.RasterCalculator_sa(expression, weighted_distance)


In [ ]:
# 计算水分平衡
water_balance = "water_balance.tif"
expression = "Raster('{}') - Raster('{}')".format(precipitation, evapotranspiration)
arcpy.gp.RasterCalculator_sa(expression, water_balance)


In [ ]:
# 标准化坡度
normalized_slope = "normalized_slope.tif"
min_slope = arcpy.GetRasterProperties_management(slope, "MINIMUM").getOutput(0)
max_slope = arcpy.GetRasterProperties_management(slope, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(slope, min_slope, max_slope, min_slope)
arcpy.gp.RasterCalculator_sa(expression, normalized_slope)

# 标准化流积量
normalized_flow_accumulation = "normalized_flow_accumulation.tif"
min_flow = arcpy.GetRasterProperties_management(flow_accumulation, "MINIMUM").getOutput(0)
max_flow = arcpy.GetRasterProperties_management(flow_accumulation, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(flow_accumulation, min_flow, max_flow, min_flow)
arcpy.gp.RasterCalculator_sa(expression, normalized_flow_accumulation)

# 标准化土壤类型
normalized_soil_type = "normalized_soil_type.tif"
remap = RemapValue([[1, 0], [2, 0.33], [3, 0.67], [4, 1]])
arcpy.gp.Reclassify_sa(soil_type, "VALUE", remap, normalized_soil_type)

# 标准化距离和高程差的加权组合
normalized_weighted_distance = "normalized_weighted_distance.tif"
min_distance = arcpy.GetRasterProperties_management(weighted_distance, "MINIMUM").getOutput(0)
max_distance = arcpy.GetRasterProperties_management(weighted_distance, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(weighted_distance, min_distance, max_distance, min_distance)
arcpy.gp.RasterCalculator_sa(expression, normalized_weighted_distance)

# 标准化水分平衡
normalized_water_balance = "normalized_water_balance.tif"
min_water_balance = arcpy.GetRasterProperties_management(water_balance, "MINIMUM").getOutput(0)
max_water_balance = arcpy.GetRasterProperties_management(water_balance, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(water_balance, min_water_balance, max_water_balance, min_water_balance)
arcpy.gp.RasterCalculator_sa(expression, normalized_water_balance)


In [ ]:
# 综合计算灌溉条件
irrigation_condition = "irrigation_condition.tif"
expression = ("0.2 * Raster('{}') + 0.2 * Raster('{}') + 0.2 * Raster('{}') + "
              "0.2 * Raster('{}') + 0.2 * Raster('{}')").format(
    normalized_slope, normalized_flow_accumulation, normalized_soil_type,
    normalized_weighted_distance, normalized_water_balance)
arcpy.gp.RasterCalculator_sa(expression, irrigation_condition)


In [ ]:
# 分类灌溉条件
classified_irrigation_condition = "classified_irrigation_condition.tif"
remap = RemapRange([[0, 0.25, 1], [0.25, 0.5, 2], [0.5, 0.75, 3], [0.75, 1, 4]])
arcpy.gp.Reclassify_sa(irrigation_condition, "VALUE", remap, classified_irrigation_condition)


In [ ]:
# 清理
arcpy.CheckInExtension("Spatial")


# 灌溉水能力

In [1]:
import arcpy
from arcpy.sa import *
import os

In [18]:


def set_environment(workspace):
    arcpy.env.workspace = workspace
    arcpy.env.overwriteOutput = True
    # 设置单进程模式
    arcpy.env.parallelProcessingFactor = "1"

def process_dem(dem):
    filled_dem = Fill(dem)
    filled_dem.save("filled_dem")
    slope = Slope(filled_dem, "DEGREE")
    slope.save("slope")
    flow_direction = FlowDirection(filled_dem)
    flow_direction.save("flow_direction")
    flow_accumulation = FlowAccumulation(flow_direction)
    flow_accumulation.save("flow_accumulation")
    return filled_dem, slope, flow_direction, flow_accumulation

def calculate_water_source_accessibility(rivers_shp, canals_shp, dem):
    combined_features = arcpy.management.Merge([rivers_shp, canals_shp], "combined_features")
    distance_to_water = EucDistance(combined_features)
    cell_size = arcpy.management.GetRasterProperties(dem, "CELLSIZEX").getOutput(0)
    combined_raster = 'combined_features'
    arcpy.conversion.FeatureToRaster(combined_features, "FID", "combined_features", cell_size)
    elevation_difference = EucAllocation(
        in_source_data=dem,
        in_value_raster=combined_raster,
        cell_size=cell_size,
        source_field="VALUE",
        distance_method="PLANAR")
    weighted_distance = 0.7 * distance_to_water + 0.3 * elevation_difference
    return weighted_distance


def calculate_water_source_accessibility(rivers_shp, canals_shp, dem):
    # Merge rivers and canals shapefiles
    combined_features = arcpy.management.Merge([rivers_shp, canals_shp], "in_memory/combined_features")

    # Convert combined features to raster
    combined_raster = "in_memory/combined_raster"
    cell_size = arcpy.management.GetRasterProperties(dem, "CELLSIZEX").getOutput(0)
    arcpy.conversion.FeatureToRaster(combined_features, "OBJECTID", combined_raster, cell_size)

    # Calculate Euclidean distance to the water sources
    distance_to_water = EucDistance(combined_raster)

    # Calculate Euclidean allocation
    elevation_difference = EucAllocation(
        in_source_data=dem,
        in_value_raster=combined_raster,
        cell_size=cell_size,
        source_field="VALUE",
        distance_method="PLANAR"
    )
    
    # Calculate weighted distance
    weighted_distance = 0.7 * distance_to_water + 0.3 * elevation_difference
    
    return weighted_distance



def normalize_raster(raster):
    min_val = float(arcpy.GetRasterProperties_management(raster, "MINIMUM").getOutput(0))
    max_val = float(arcpy.GetRasterProperties_management(raster, "MAXIMUM").getOutput(0))
    return (Raster(raster) - min_val) / (max_val - min_val)

def reclassify_soil(soil_type):
    soil_remap = RemapValue([
        [1, 0.2],   # Sandy soil (low water retention)
        [2, 0.4],   # Sandy loam
        [3, 0.6],   # Loam
        [4, 0.8],   # Clay loam
        [5, 1.0]    # Clay soil (high water retention)
    ])
    return Reclassify(soil_type, "VALUE", soil_remap)

def calculate_irrigation_capacity(slope, flow_accumulation, water_source_accessibility, water_balance):
    norm_slope = normalize_raster(slope)
    norm_flow_acc = normalize_raster(flow_accumulation)
    norm_accessibility = normalize_raster(water_source_accessibility)
    norm_water_balance = normalize_raster(water_balance)
    
    
    irrigation_capacity = (0.15 * (1 - norm_slope) +  # Invert slope for irrigation suitability
                           0.15 * norm_flow_acc +
                           0.2 * (1 - norm_accessibility) +  # Invert accessibility
                           0.2 * norm_water_balance
                       )
    
    return irrigation_capacity

In [ ]:

workspace = r"C:\Users\Runker\Desktop\DEM_test\drainwater.gdb"
set_environment(workspace)

# Input data
dem = "DEM"
rivers_shp = "river"
canals_shp = "vally"
precipitation = "PRE2022_mean"


# Process DEM
process_dem(dem)
filled_dem, slope, flow_direction, flow_accumulation = process_dem(dem)



In [ ]:
# Calculate water source accessibility
water_source_accessibility = calculate_water_source_accessibility(rivers_shp, canals_shp, filled_dem)
water_source_accessibility.save("water_source_accessibility")




In [19]:
# Calculate irrigation capacity
irrigation_capacity = calculate_irrigation_capacity(
    slope, flow_accumulation, water_source_accessibility, precipitation)
irrigation_capacity.save("irrigation_capacity")

# Classify irrigation capacity
classified_irrigation_capacity = Reclassify(irrigation_capacity, "Value",
                                            RemapRange([[0, 0.2, 1],
                                                        [0.2, 0.4, 2],
                                                        [0.4, 0.6, 3],
                                                        [0.6, 0.8, 4],
                                                        [0.8, 1, 5]]))
classified_irrigation_capacity.save("classified_irrigation_capacity")

print("灌溉能力计算完成。")

灌溉能力计算完成。
